In [7]:
import ee
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime
import os
import io
from googleapiclient.http import MediaIoBaseDownload

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'automate2-428111-c5d352d7461e.json'

# Initialize Earth Engine with service account credentials
credentials = ee.ServiceAccountCredentials('automate2acc@automate2-428111.iam.gserviceaccount.com', SERVICE_ACCOUNT_FILE)
ee.Initialize(credentials)

# Authentication for Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
drive_credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=drive_credentials)

print("Authentication successful")


Authentication successful


In [4]:
df = pd.read_csv('saved_attribute_table.csv')  # Update the path to your CSV file

In [5]:
import pandas as pd
from datetime import datetime

# Assuming 'df' is your dataframe and 'first_gap' is already in datetime format
df['first_gap'] = pd.to_datetime(df['first_gap'], format='%Y-%m')

# Calculate end_date as two years after first_gap
df['end_date'] = df['first_gap'] + pd.DateOffset(years=2)

# Ensure end_date does not go beyond today's date
df['end_date'] = df['end_date'].apply(lambda x: min(x, pd.Timestamp(datetime.now())))

# Display the dataframe to verify the new column
display(df)


,fid,event,last_alive,first_stan,last_stand,first_gap,notes,confirmed,centroid_x,centroid_y,area,end_date
0,22,standing&b/u,NaN,2016-06,2017-12,2018-06-01,NaN,checked,556651.360,9533727.893,105.412,2020-06-01 00:00:00.000000
1,23,standing&b/u,NaN,2016-06,2017-06,2017-12-01,NaN,checked,555897.816,9533599.821,156.257,2019-12-01 00:00:00.000000
2,46,standing&b/u,2016-06,2016-12,2023-04,2023-10-01,very few branches and mostly gap now,checked,561176.381,9523245.382,106.495,2024-08-06 12:32:05.481854
3,62,standing&b/u,2018-06,2018-12,2019-06,2020-06-01,NaN,checked,563785.032,9527067.637,207.746,2022-06-01 00:00:00.000000
4,66,b/u,2023-09,NaN,NaN,2023-10-01,NaN,checked,563329.777,9527728.359,265.294,2024-08-06 12:32:05.481854
...,...,...,...,...,...,...,...,...,...,...,...,...
102,936,?b/u,2022-10,NaN,NaN,2022-11-01,NaN,?,557663.011,9544779.686,41.274,2024-08-06 12:32:05.481854
103,938,b/u,2022-10,NaN,NaN,2022-11-01,NaN,checked,557568.600,9544912.789,122.074,2024-08-06 12:32:05.481854
104,954,standing&b/u,2019-06,2019-12,2020-06,2020-11-01,NaN,checked,558675.979,9548226.623,78.080,2022-11-01 00:00:00.000000
105,958,b/u,2022-03,NaN,NaN,2022-06-01,NaN,checked,559284.896,9548405.319,85.460,2024-06-01 00:00:00.000000


In [32]:
df

,fid,event,last_alive,first_standing,last_standing,first_gap,notes,confirmed,centroid_x,centroid_y,end_date
0,781,standing&b/u,2018-06,2019-06,2023-04,2023-08-01,NaN,checked,215042.100906,9.548664e+06,2024-07-12 15:43:31.237278
1,785,standing&b/u,2019-12,2020-06,2020-09,2020-12-01,NaN,checked,217941.875568,9.551028e+06,2022-12-01 00:00:00.000000
2,786,b/u,2019-06,NaN,NaN,2020-09-01,NaN,cheched,218089.543617,9.548737e+06,2022-09-01 00:00:00.000000
3,787,standing&b/u,2018-06,2018-12,2021-12,2022-11-01,NaN,checked,215374.010965,9.549249e+06,2024-07-12 15:43:31.238277
4,789,standing&b/u,2018-06,2019-12,2021-12,2022-03-01,NaN,checked,210882.484226,9.543659e+06,2024-03-01 00:00:00.000000


In [59]:
# Function to list all files and folders
def list_all_files_and_folders():
    query = "trashed=false"
    results = service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)').execute()
    items = results.get('files', [])

    if not items:
        print('No files or folders found.')
    else:
        print('Files and folders:')
        for item in items:
            print(f"{item['name']} ({item['id']})")

# List all files and folders
list_all_files_and_folders()

Files and folders:
gap84-2023-12-02.tif (1mpSPdUfmyVteGGzAPzUFRRF0oZLyiCuG)
gap84-2023-12-14.tif (1WQaZTnjK1vUOVOWlZKjMWDHf5UbEkbVf)
gap84-2023-11-20.tif (1LhGsv5ij7WlAnae4OkId_NWtn3RTqVWO)
gap84-2023-11-08.tif (1rJHrqAamRRQHb9JUvc4fmiEininvtYeR)
gap84-2023-10-27.tif (1ORsARknuNIwjFc-0XmhRmOgK21ofWWIS)
gap84-2023-10-15.tif (1PRy-YXvUOziMPiDOV7fd1Tl5qJsJRqxp)
gap84-2023-09-21.tif (1uWh3Crqk-2buj8epCCMoOTnckcxAu2Yz)
gap84-2023-10-03.tif (18gOt8GBRTKAKiWXfh3p9YLzbNN4fO_Js)
gap84-2023-09-09.tif (1K8Ac7yJHooS6Qi4lfp2RexzLpBdWoUhZ)
gap84-2023-08-28.tif (1nRVW1omFgDX02Ud5veKbKdG5QH1fj7R1)
gap84-2023-08-16.tif (1NaBhgwTZ9Xa2sWzz-vr8KG7DyjlELVkg)
gap84-2023-08-04.tif (1fjGBBwFcEe9zzDxfO-N6DeqysMGP-Tx-)
gap84-2023-06-29.tif (100JSyy32saB788n3Djd32g5ZTeXmtKkh)
gap84-2023-07-23.tif (1zVvIEI1H9tB_wt1XcQhugeQAETDHPCZS)
gap84-2023-06-05.tif (1IzdLTE2owlP22Tah83y3dkhntSbn5u77)
gap84-2023-05-24.tif (1mDsm9rnP8RPOkWmyVt6hkXC0HhU7QWFy)
gap84-2023-05-12.tif (1s7JRaLm_DHD_RUMGMN24VX3OTfuJDEBE)
gap84-2023-0

In [54]:
def delete_all_files():
    page_token = None

    while True:
        # List files in batches of 100
        results = service.files().list(
            spaces='drive',
            fields='nextPageToken, files(id, name)',
            pageToken=page_token,
            pageSize=100  # Maximum page size is 100
        ).execute()
        
        items = results.get('files', [])
        
        if not items:
            print('No more files found.')
            break

        # Delete files
        for item in items:
            service.files().delete(fileId=item['id']).execute()
            print(f"Deleted {item['name']} ({item['id']})")

        # Update the page token
        page_token = results.get('nextPageToken')
        if not page_token:
            break

# Delete all files in the service account's Google Drive
delete_all_files()


No more files found.


## Function for Old coordinate system

In [36]:

# Function to export Sentinel-1 images
def export_sentinel1_images(start_date, end_date, interval_days, easting, northing, export_region, iteration):
    point = ee.Geometry.Point([easting, northing], 'EPSG:32720')  # Assuming UTM zone 20S
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(point) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .select(['VV', 'VH'])

    start_date_obj = ee.Date(start_date)
    end_date_obj = ee.Date(end_date)
    num_images = end_date_obj.difference(start_date_obj, 'day').divide(interval_days).ceil().toInt()

    for i in range(num_images.getInfo()):
        current_start_date = start_date_obj.advance(i * interval_days, 'day')
        current_end_date = current_start_date.advance(interval_days, 'day')

        # Ensure the end date does not exceed the specified end_date
        if current_end_date.millis().gt(end_date_obj.millis()).getInfo():
            current_end_date = end_date_obj

        sentinel1_subset = sentinel1.filterDate(current_start_date, current_end_date).first()

        # Check if the image is valid
        if sentinel1_subset.getInfo() is None:
            print(f"No image found for date range: {current_start_date.format('YYYY-MM-dd').getInfo()} to {current_end_date.format('YYYY-MM-dd').getInfo()}")
            continue

        image_date = sentinel1_subset.date().format('YYYY-MM-dd').getInfo()  # Get the actual date of the image

        # Ensure the description is valid
        export_description = f'gap{iteration}-{current_start_date.format("YYYY-MM-dd").getInfo()}'

        task = ee.batch.Export.image.toDrive(
            image=sentinel1_subset.select('VV'),
            description=export_description,
            scale=10,
            region=export_region,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        task.start()

        print(f'Exporting image taken on: {image_date} with description {export_description}')

subset_df = df.iloc[80:85]

# Iterate through the selected rows
for index, row in subset_df.iterrows():
    easting = row['centroid_x']
    northing = row['centroid_y']
    start_date = row['first_gap']
    end_date = row['end_date']
    interval_days = 6  # Set interval to 6 days

    # Define the export region around the point (buffered by 250 meters and then bounds)
    point = ee.Geometry.Point([easting, northing], 'EPSG:32720')
    export_region = point.buffer(250).bounds()  # Adjust buffer size as needed

    # Debug print statements to ensure correct iteration
    print(f'Starting export for gap {index + 1}')
    export_sentinel1_images(start_date, end_date, interval_days, easting, northing, export_region, index + 1)
    print(f'Completed export for gap {index + 1}')

print("Export tasks for all rows started.")


Starting export for gap 1
No image found for date range: 2023-08-01 to 2023-08-07
Exporting image taken on: 2023-08-08 with description gap1-2023-08-07
No image found for date range: 2023-08-13 to 2023-08-19
Exporting image taken on: 2023-08-20 with description gap1-2023-08-19


KeyboardInterrupt: 

## Function for 81-85 new coordinate system

In [43]:
for index, row in df.iterrows():
    print(index)

0
1
2
3
4


In [56]:
# Function to export Sentinel-1 images
def export_sentinel1_images(start_date, end_date, interval_days, easting, northing, export_region, iteration):
    point = ee.Geometry.Point([easting, northing], 'EPSG:29181')  # Assuming UTM zone 21S
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(point) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .select(['VV', 'VH'])

    start_date_obj = ee.Date(start_date)
    end_date_obj = ee.Date(end_date)
    num_images = end_date_obj.difference(start_date_obj, 'day').divide(interval_days).ceil().toInt()

    for i in range(num_images.getInfo()):
        current_start_date = start_date_obj.advance(i * interval_days, 'day')
        current_end_date = current_start_date.advance(interval_days, 'day')

        # Ensure the end date does not exceed the specified end_date
        if current_end_date.millis().gt(end_date_obj.millis()).getInfo():
            current_end_date = end_date_obj

        sentinel1_subset = sentinel1.filterDate(current_start_date, current_end_date).first()

        # Check if the image is valid
        if sentinel1_subset.getInfo() is None:
            print(f"No image found for date range: {current_start_date.format('YYYY-MM-dd').getInfo()} to {current_end_date.format('YYYY-MM-dd').getInfo()}")
            continue

        image_date = sentinel1_subset.date().format('YYYY-MM-dd').getInfo()  # Get the actual date of the image

        # Ensure the description is valid
        export_description = f'gap{iteration + 81}-{current_start_date.format("YYYY-MM-dd").getInfo()}'  # Starting at gap81

        task = ee.batch.Export.image.toDrive(
            image=sentinel1_subset.select('VV'),
            description=export_description,
            scale=10,
            region=export_region,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        task.start()

        print(f'Exporting image taken on: {image_date} with description {export_description}')

# Iterate through the selected rows
for index, row in df.iterrows():
    easting = row['centroid_x']
    northing = row['centroid_y']
    start_date = row['first_gap']
    end_date = row['end_date']
    interval_days = 6  # Set interval to 6 days

    # Define the export region around the point (buffered by 250 meters and then bounds)
    point = ee.Geometry.Point([easting, northing], 'EPSG:32720')
    export_region = point.buffer(250).bounds()  # Adjust buffer size as needed

    # Debug print statements to ensure correct iteration
    print(f'Starting export for gap {index + 81}')  # Starting at gap81
    export_sentinel1_images(start_date, end_date, interval_days, easting, northing, export_region, index)
    print(f'Completed export for gap {index + 81}')

print("Export tasks for all rows started.")


Starting export for gap 81
No image found for date range: 2023-08-01 to 2023-08-07
Exporting image taken on: 2023-08-08 with description gap81-2023-08-07
No image found for date range: 2023-08-13 to 2023-08-19
Exporting image taken on: 2023-08-20 with description gap81-2023-08-19
No image found for date range: 2023-08-25 to 2023-08-31
Exporting image taken on: 2023-09-01 with description gap81-2023-08-31
No image found for date range: 2023-09-06 to 2023-09-12
Exporting image taken on: 2023-09-13 with description gap81-2023-09-12
No image found for date range: 2023-09-18 to 2023-09-24
Exporting image taken on: 2023-09-25 with description gap81-2023-09-24
No image found for date range: 2023-09-30 to 2023-10-06
Exporting image taken on: 2023-10-07 with description gap81-2023-10-06
No image found for date range: 2023-10-12 to 2023-10-18
Exporting image taken on: 2023-10-19 with description gap81-2023-10-18
No image found for date range: 2023-10-24 to 2023-10-30
Exporting image taken on: 20

In [57]:
# Define the download directory
download_directory = 'downloaded_files_gaps81-85'
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Function to download and delete all files in batches
def download_and_delete_all_files():
    page_token = None

    while True:
        # List files in the service account's Google Drive
        results = service.files().list(
            spaces='drive',
            fields='nextPageToken, files(id, name)',
            pageToken=page_token
        ).execute()
        
        items = results.get('files', [])
        
        if not items:
            print('No more files found.')
            break

        # Download files
        for item in items:
            request = service.files().get_media(fileId=item['id'])
            file_path = os.path.join(download_directory, item['name'])
            fh = io.FileIO(file_path, 'wb')
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloading {item['name']} ({item['id']}): {int(status.progress() * 100)}%")

        # Delete files
        for item in items:
            service.files().delete(fileId=item['id']).execute()
            print(f"Deleted {item['name']} ({item['id']})")

        # Update the page token
        page_token = results.get('nextPageToken')
        if not page_token:
            break

# Run the combined function to download and delete all files
download_and_delete_all_files()
print("All files have been downloaded and deleted.")


KeyboardInterrupt: 

In [23]:
# Define the download directory
download_directory = 'downloaded_files_gaps81-85'
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Function to download and delete all files in batches
def download_and_delete_all_files():
    page_token = None

    while True:
        # List files in the service account's Google Drive
        results = service.files().list(
            spaces='drive',
            fields='nextPageToken, files(id, name)',
            pageToken=page_token
        ).execute()
        
        items = results.get('files', [])
        
        if not items:
            print('No more files found.')
            break

        # Download files
        for item in items:
            request = service.files().get_media(fileId=item['id'])
            file_path = os.path.join(download_directory, item['name'])
            fh = io.FileIO(file_path, 'wb')
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloading {item['name']} ({item['id']}): {int(status.progress() * 100)}%")

        # Delete files
        for item in items:
            service.files().delete(fileId=item['id']).execute()
            print(f"Deleted {item['name']} ({item['id']})")

        # Update the page token
        page_token = results.get('nextPageToken')
        if not page_token:
            break

# Run the combined function to download and delete all files
download_and_delete_all_files()
print("All files have been downloaded and deleted.")


Deleted gap85-2024-02-25.tif (17Q28lNq0T7k-KCGAdg9aaiYTGEx2naI9)
Deleted gap85-2024-02-13.tif (1QJ1WcU0_PJ-39VZmPQS_OAuggroWzXGU)
Deleted gap85-2024-02-01.tif (1x9gwMCM49KVAPFhiJPhcE9U46a46aSUI)
Deleted gap85-2024-01-08.tif (1C_XHMKHLqU_T5c7dUWKvplWI-Rs5mQkP)
Deleted gap85-2024-01-20.tif (1tKo5QaPEeDR-jg-ymIIBBwYsw2DEBXXm)
Deleted gap85-2023-12-27.tif (1QW6tCeUe9G68XNynvLsS1cuLXrJiRTEK)
Deleted gap85-2023-12-15.tif (1L9q9sZtY9_ee5KG3CP8Dk4bjvPzOXSvg)
Deleted gap85-2023-12-03.tif (13cpdipFsuP4UrVvHmiZilksTm9N_EGoA)
Deleted gap85-2023-11-21.tif (17g_3iEeIKgTf_lILAfEOG38yqsBzLmNE)
Deleted gap85-2023-11-09.tif (1mSyHw0Vd9Tk_4kjJw4_lHSVzFtIw5_og)
Deleted gap85-2023-10-28.tif (17f-lfrYiThataH8UickNcuJH5pDmoiSm)
Deleted gap85-2023-10-16.tif (1Ti4cm57YGKwQ30Fu6QPlodYHsfeARouE)
Deleted gap85-2023-09-22.tif (1eYioa7yyYT599DtV2wqDBy8lxXTngKnM)
Deleted gap85-2023-10-04.tif (1EyrUCWkOkn2G-cQdA3tJoGHtaiu-kivf)
Deleted gap85-2023-09-10.tif (1dx2pmSFPmtgxgk3yoIFIriD3bU7wRXez)
Deleted gap85-2023-08-17.

In [20]:
def list_oldest_files(count=5):
    page_token = None
    all_files = []

    while True:
        results = service.files().list(
            spaces='drive',
            fields='nextPageToken, files(id, name, createdTime)',
            orderBy='createdTime',
            pageToken=page_token
        ).execute()
        
        items = results.get('files', [])
        all_files.extend(items)
        
        page_token = results.get('nextPageToken')
        if not page_token:
            break

    # Sort files by createdTime in ascending order
    all_files = sorted(all_files, key=lambda x: x['createdTime'])
    
    # Print the oldest files
    if not all_files:
        print('No files found.')
    else:
        print(f'Total number of files: {len(all_files)}')
        print('Oldest files:')
        for item in all_files[:count]:
            print(f"{item['name']} (ID: {item['id']}, Created Time: {item['createdTime']})")

# List the 5 oldest files in the service account's Google Drive
list_oldest_files(count=5)


Total number of files: 251
Oldest files:
gap81-2023-08-07.tif (ID: 1sF2LkbvX5Z4LaRnOBIAlAQkhhl9ML5qV, Created Time: 2024-07-12T11:28:21.095Z)
gap81-2023-08-19.tif (ID: 1gMgy6YA7wUaKn9LgTAViVpVTsL90nuBT, Created Time: 2024-07-12T11:28:29.013Z)
gap81-2023-09-12.tif (ID: 1S6BRdcunRSQQBvFYdo_wcPykNiAj7S8X, Created Time: 2024-07-12T11:28:38.967Z)
gap81-2023-08-31.tif (ID: 1aTFgba9A4iD1yGCNmw7JYDQVVRtnLXcK, Created Time: 2024-07-12T11:28:41.409Z)
gap81-2023-09-24.tif (ID: 1Z4DIBIcMmtDWaWT_UPuGI36_Hl8Qd_La, Created Time: 2024-07-12T11:28:48.063Z)


In [22]:
def list_newest_files(count=5):
    page_token = None
    all_files = []

    while True:
        results = service.files().list(
            spaces='drive',
            fields='nextPageToken, files(id, name, createdTime)',
            orderBy='createdTime desc',
            pageToken=page_token
        ).execute()
        
        items = results.get('files', [])
        all_files.extend(items)
        
        page_token = results.get('nextPageToken')
        if not page_token:
            break

    # Sort files by createdTime in descending order
    all_files = sorted(all_files, key=lambda x: x['createdTime'], reverse=True)
    
    # Print the newest files
    if not all_files:
        print('No files found.')
    else:
        print(f'Total number of files: {len(all_files)}')
        print('Newest files:')
        for item in all_files[:count]:
            print(f"{item['name']} (ID: {item['id']}, Created Time: {item['createdTime']})")

# List the 5 newest files in the service account's Google Drive
list_newest_files(count=5)


Total number of files: 316
Newest files:
gap85-2024-02-25.tif (ID: 17Q28lNq0T7k-KCGAdg9aaiYTGEx2naI9, Created Time: 2024-07-12T11:57:16.979Z)
gap85-2024-02-13.tif (ID: 1QJ1WcU0_PJ-39VZmPQS_OAuggroWzXGU, Created Time: 2024-07-12T11:57:13.421Z)
gap85-2024-02-01.tif (ID: 1x9gwMCM49KVAPFhiJPhcE9U46a46aSUI, Created Time: 2024-07-12T11:57:10.819Z)
gap85-2024-01-08.tif (ID: 1C_XHMKHLqU_T5c7dUWKvplWI-Rs5mQkP, Created Time: 2024-07-12T11:56:59.904Z)
gap85-2024-01-20.tif (ID: 1tKo5QaPEeDR-jg-ymIIBBwYsw2DEBXXm, Created Time: 2024-07-12T11:56:59.491Z)


In [26]:
import os
import shutil

# Define the path to the downloaded files directory
download_directory = 'downloaded_files_gaps81-85'

# Get a list of all files in the download directory
files = os.listdir(download_directory)

# Create a dictionary to track the folders
folders = {}

# Loop through each file in the directory
for file in files:
    # Skip directories
    if os.path.isdir(os.path.join(download_directory, file)):
        continue
    
    # Extract the gap number from the file name (assuming the format 'gapX-YYYY-MM-DD')
    gap_number = file.split('-')[0]
    
    # Create the folder name
    folder_name = gap_number
    
    # Create the folder if it doesn't exist
    folder_path = os.path.join(download_directory, folder_name)
    if folder_name not in folders:
        os.makedirs(folder_path, exist_ok=True)
        folders[folder_name] = folder_path
    
    # Move the file to the corresponding folder
    shutil.move(os.path.join(download_directory, file), os.path.join(folder_path, file))

print("Files have been sorted into their respective folders.")


Files have been sorted into their respective folders.


In [25]:
import rasterio
import numpy as np

def compare_tif_images(image1_path, image2_path):
    try:
        # Open the first image
        with rasterio.open(image1_path) as src1:
            image1 = src1.read()
        
        # Open the second image
        with rasterio.open(image2_path) as src2:
            image2 = src2.read()
        
        # Compare the pixel values of the two images
        if np.array_equal(image1, image2):
            return True
        else:
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

# Paths to the two .tif images
image1_path = 'downloaded_files_gaps81-85/gap83-2021-03-30.tif'
image2_path = 'downloaded_files_gaps81-85/gap83-2021-04-05.tif'

# Check if the images are the same
are_images_same = compare_tif_images(image1_path, image2_path)

if are_images_same:
    print("The images are the same.")
else:
    print("The images are different.")


The images are different.


In [9]:
import ee
import pandas as pd
from google.oauth2 import service_account
from datetime import datetime
from pyproj import Proj, transform

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'automate2-428111-c5d352d7461e.json'

# Initialize Earth Engine with service account credentials
credentials = ee.ServiceAccountCredentials('automate2acc@automate2-428111.iam.gserviceaccount.com', SERVICE_ACCOUNT_FILE)
ee.Initialize(credentials)

print("Authentication successful")

# Define UTM projection (example for UTM zone 20S, EPSG:32720)
utm_proj = Proj(proj='utm', zone=20, south=True, ellps='WGS84')
wgs84_proj = Proj(proj='latlong', datum='WGS84')

# Example UTM coordinates
utm_coords = [
    (556651.360, 9533727.893),
    (555897.816, 9533599.821),
    (561176.381, 9523245.382),
    (563785.032, 9527067.637),
    (563329.777, 9527728.359)
]

# Convert UTM to Lat/Lon
latlon_coords = [transform(utm_proj, wgs84_proj, x, y) for x, y in utm_coords]

# Define the region of interest (ROI) using the converted coordinates
roi = ee.Geometry.MultiPoint(latlon_coords)

# Define the date range
start_date = '2018-01-01'
end_date = '2023-12-31'

# Load the Sentinel-1 ImageCollection
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .select(['VV', 'VH'])

# Get the acquisition dates
acquisition_dates = sentinel1.aggregate_array('system:time_start').getInfo()
acquisition_dates = [datetime.utcfromtimestamp(date / 1000).strftime('%Y-%m-%d') for date in acquisition_dates]

# Sort dates
acquisition_dates.sort()

# Print the acquisition dates
print('Acquisition Dates:', acquisition_dates)

# Calculate the interval between acquisitions
intervals = []
for i in range(1, len(acquisition_dates)):
    date1 = datetime.strptime(acquisition_dates[i-1], '%Y-%m-%d')
    date2 = datetime.strptime(acquisition_dates[i], '%Y-%m-%d')
    interval = (date2 - date1).days
    intervals.append(interval)

# Print the intervals
print('Intervals between acquisitions (days):', intervals)

# Convert to DataFrame for better visualization
df = pd.DataFrame({
    'Acquisition Date': acquisition_dates,
    'Interval (days)': [None] + intervals  # Add None for the first date where there is no interval
})

# Display the DataFrame
df.T

Authentication successful


C:\Users\olive\AppData\Local\Temp\ipykernel_35996\388165678.py:30: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  latlon_coords = [transform(utm_proj, wgs84_proj, x, y) for x, y in utm_coords]


Acquisition Dates: ['2018-01-11', '2018-01-23', '2018-02-04', '2018-02-16', '2018-02-28', '2018-03-12', '2018-03-24', '2018-04-05', '2018-04-17', '2018-04-29', '2018-05-11', '2018-05-23', '2018-06-04', '2018-06-16', '2018-06-28', '2018-07-10', '2018-07-22', '2018-08-03', '2018-08-15', '2018-09-08', '2018-09-20', '2018-10-02', '2018-10-14', '2018-10-26', '2018-11-07', '2018-11-19', '2018-12-01', '2018-12-13', '2018-12-25', '2019-01-06', '2019-01-18', '2019-01-30', '2019-02-11', '2019-02-23', '2019-03-07', '2019-03-19', '2019-03-31', '2019-04-12', '2019-04-24', '2019-05-06', '2019-05-18', '2019-05-30', '2019-06-11', '2019-06-23', '2019-07-05', '2019-07-17', '2019-07-29', '2019-08-10', '2019-08-22', '2019-09-03', '2019-09-15', '2019-09-27', '2019-10-09', '2019-10-21', '2019-11-02', '2019-11-14', '2019-11-26', '2019-12-08', '2019-12-20', '2020-01-01', '2020-01-13', '2020-01-25', '2020-02-06', '2020-02-18', '2020-03-01', '2020-03-13', '2020-03-25', '2020-04-06', '2020-04-18', '2020-04-30', 

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
Acquisition Date,2018-01-11,2018-01-23,2018-02-04,2018-02-16,2018-02-28,2018-03-12,2018-03-24,2018-04-05,2018-04-17,2018-04-29,...,2023-09-06,2023-09-18,2023-09-30,2023-10-12,2023-10-24,2023-11-05,2023-11-17,2023-11-29,2023-12-11,2023-12-23
Interval (days),NaN,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0


In [8]:


def get_sentinel1_metadata(start_date, end_date, easting, northing):
    # Define the point geometry using the provided coordinates
    point = ee.Geometry.Point([easting, northing], 'EPSG:32720')
    
    # Filter Sentinel-1 image collection
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    
    # Function to extract relevant metadata from each image
    def extract_metadata(image):
        orbit_number = image.get('relativeOrbitNumber_start')
        orbit_direction = image.get('orbitProperties_pass')
        look_angle = image.get('angle')
        acquisition_time = image.get('system:time_start')
        return ee.Feature(None, {
            'orbit_number': orbit_number,
            'orbit_direction': orbit_direction,
            'look_angle': look_angle,
            'acquisition_time': acquisition_time
        })
    
    # Map the function over the image collection to extract metadata
    metadata = sentinel1.map(extract_metadata)
    
    # Get the metadata as a list of features
    metadata_list = metadata.getInfo()['features']
    
    # Print metadata
    for feature in metadata_list:
        properties = feature['properties']
        print(f"Orbit Number: {properties['orbit_number']}, "
              f"Orbit Direction: {properties['orbit_direction']}, "
              f"Look Angle: {properties['look_angle']}, "
              f"Acquisition Time: {properties['acquisition_time']}")
        
    return metadata_list

# Example usage
start_date = '2023-01-01'
end_date = '2023-12-31'
easting = 563329.777
northing = 9527728.359

metadata_list = get_sentinel1_metadata(start_date, end_date, easting, northing)


Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1673257701000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1674294500000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1676368100000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1677404900000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1678441700000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1679478500000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1680515275000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1681552101000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1682588901000
Orbit Number: 83, Orbit Direction: DESCENDING, Look Angle: None, Acquisition Time: 1683625702000
Orbit Number: 83, Orbit Direct